#### This code Tests following configuration:

Entity Representation: Centroid of Context words + Words in Entity

Type Representation: Type representation learned on training corpus

Ranking Score: Group Average

Word Representation: Trained word vectors on corpus initialized with pretrained glove vector embeddings

Note: Gives the same results as cosine

In [1]:
import os, pickle
from operator import itemgetter
import numpy as np

In [2]:
entity_ranking_op_folder = '../../Data/output'
type_rep_folder = 'TypeRep/type_word/train'
entity_rep_test_folder = 'EntityRep/test'
entity_rep_file_name = 'doc_role_entity_context_word_centroid.p' 
type_rep_file_name = 'tag_vec_dict.p'

entity_rep_file = os.path.join(entity_ranking_op_folder, entity_rep_test_folder, entity_rep_file_name)
type_rep_file = os.path.join(entity_ranking_op_folder, type_rep_folder, type_rep_file_name)

In [3]:
tag_list = ['LOC_Event', 'LOC_Accused', 'LOC_Victim', 'LOC_Others', 'ORG_Accused', 'ORG_Victim', 'ORG_Others', 'PER_Victim', 'PER_Others', 'PER_Accused']
per_tag_list = ['PER_Victim', 'PER_Others', 'PER_Accused']
loc_tag_list = ['LOC_Event', 'LOC_Accused', 'LOC_Victim', 'LOC_Others']
org_tag_list = ['ORG_Accused', 'ORG_Victim', 'ORG_Others']

In [4]:
entity_rep_doc_role_context_dict = pickle.load(open(entity_rep_file, 'rb'))
type_rep_dict = pickle.load(open(type_rep_file, 'rb'))

In [5]:
def group_average(entity, role):
    
    size = 300
    num_of_words_in_entity = 1
    num_of_words_in_role = 1
    total_number_of_words = num_of_words_in_entity + num_of_words_in_role
    
    sum_of_entity_type_word_vec = np.zeros(size)
    
    entity = entity/np.linalg.norm(entity)
    sum_of_entity_type_word_vec = np.add(sum_of_entity_type_word_vec, entity)
        
    role = role/np.linalg.norm(role)
    sum_of_entity_type_word_vec = np.add(sum_of_entity_type_word_vec, role)
    
    dot_product = np.dot(sum_of_entity_type_word_vec, sum_of_entity_type_word_vec)
    group_avg = float(dot_product - (total_number_of_words))/(total_number_of_words * (total_number_of_words - 1))
    return group_avg 

In [6]:
precision1 = dict()
doc_count_for_tag = dict()

for tag in tag_list:
    doc_count_for_tag[tag] = 0
    precision1[tag] = 0
    

# Precision at K
K = 5
count = 0
for doc_id in entity_rep_doc_role_context_dict:
    #print(doc_id)
    num_actual_entities_with_role = dict()
    if count > 0:
        break
    per_entities = list()
    loc_entities = list()
    org_entities = list()
    doc_role_dict = entity_rep_doc_role_context_dict[doc_id]
    for role in doc_role_dict.keys():
        entities = doc_role_dict[role]
        num_actual_entities_with_role[role] = len(entities)
        if role in per_tag_list:
            for entity in entities:
                per_entities.append((entity, role))
        if role in org_tag_list:
            for entity in entities:
                org_entities.append((entity, role))
        if role in loc_tag_list:
            for entity in entities:
                loc_entities.append((entity, role))
    
    num_per_entity = len(per_entities)
    num_org_entity = len(org_entities)
    num_loc_entity = len(loc_entities)
    
    for role in per_tag_list:
        role_vector = type_rep_dict[role]
        retrivedResult = list()
        if num_actual_entities_with_role[role] != 0:
            TP = 0
            doc_count_for_tag[role] = doc_count_for_tag[role] + 1
            for entity in per_entities:
                sim = 2
                sim = group_average(entity[0][1], role_vector)
                retrivedResult.append((entity, sim))
            retrivedResult = sorted(retrivedResult,key=itemgetter(1), reverse=True)
            for i in range(min(K, num_per_entity)):
                if retrivedResult[i][0][1] == role:
                    TP = TP + 1
            #if TP ==  num_actual_entities_with_role[role]:
                #precision1[role] = precision1[role] + 1
            #else:
            precision1[role] = precision1[role] + float(TP)/min(K, num_per_entity)
                
    for role in loc_tag_list:
        role_vector = type_rep_dict[role]
        retrivedResult = list()
        if num_actual_entities_with_role[role] != 0:
            TP = 0
            doc_count_for_tag[role] = doc_count_for_tag[role] + 1
            for entity in loc_entities:
                sim = 2
                sim = group_average(entity[0][1], role_vector)
                retrivedResult.append((entity, sim))
            retrivedResult = sorted(retrivedResult,key=itemgetter(1), reverse=True)
            for i in range(min(K, num_loc_entity)):
                if retrivedResult[i][0][1] == role:
                    TP = TP + 1
            #if TP ==  num_actual_entities_with_role[role]:
             #   precision1[role] = precision1[role] + 1
            #else:
            precision1[role] = precision1[role] + float(TP)/min(K, num_loc_entity)
                
    for role in org_tag_list:
        role_vector = type_rep_dict[role]
        retrivedResult = list()
        if num_actual_entities_with_role[role] != 0:
            TP = 0
            doc_count_for_tag[role] = doc_count_for_tag[role] + 1
            for entity in org_entities:
                sim = 2
                sim = group_average(entity[0][1], role_vector)
                retrivedResult.append((entity, sim))
            retrivedResult = sorted(retrivedResult,key=itemgetter(1), reverse=True)
            for i in range(min(K, num_org_entity)):
                if retrivedResult[i][0][1] == role:
                    TP = TP + 1
            #if TP ==  num_actual_entities_with_role[role]:
             #   precision1[role] = precision1[role] + 1
            #else:
            precision1[role] = precision1[role] + float(TP)/min(K, num_org_entity)

avg_pre = 0
for tag in tag_list:
    if doc_count_for_tag[tag] > 0:
        avg_pre = avg_pre + float(precision1[tag] * 100)/doc_count_for_tag[tag]
        print(tag, float(precision1[tag] * 100)/doc_count_for_tag[tag], '%')
print('Average', avg_pre/10, '%')

LOC_Event 57.89965986394555 %
LOC_Accused 31.122448979591827 %
LOC_Victim 24.444444444444446 %
LOC_Others 49.141716566866286 %
ORG_Accused 48.67986798679868 %
ORG_Victim 37.5287356321839 %
ORG_Others 76.29310344827586 %
PER_Victim 38.978494623655926 %
PER_Others 79.7821350762527 %
PER_Accused 65.54794520547945 %
Average 50.941855182749464 %


In [7]:
from operator import itemgetter
data = [('abc', 1.21),('abc', 2.31),('abc', 1.48), ('abc',2.21)]
data = sorted(data,key=itemgetter(1), reverse=True)
data[0:4]

[('abc', 2.31), ('abc', 2.21), ('abc', 1.48), ('abc', 1.21)]